In [1]:
import os
os.chdir('/n/data1/hms/dbmi/oconnor/lab/shz311/pangenome/graph_var')

from graph_var.utils import load_graph_from_pkl
from graph_var.graph import PangenomeGraph
from graph_var.evaluating_functions import *
import pandas as pd

graph_obj_dir = '/n/data1/hms/dbmi/oconnor/lab/shz311/pangenome/Graph_objs'
vcf_dir = '/n/data1/hms/dbmi/oconnor/lab/shz311/pangenome/VCFs_update'

In [2]:
chr_list = list(range(1,23))
#chr_list = [21]

exclude_terminus = True

for chr in chr_list:
    print(f"Processing chr{chr}...")
    graph_path = f"{graph_obj_dir}/chr{chr}.pkl"
    if exclude_terminus:
        vcf_path = f"{vcf_dir}/graph_chr{chr}_no_terminus.vcf" 
    else:
        vcf_path = f"{vcf_dir}/graph_chr{chr}.vcf"
    tree_path = f"{vcf_dir}/graph_chr{chr}.tree"

    G, walks, walk_sample_names = load_graph_from_pkl(graph_path, compressed=False)
    G.write_vcf(walks, walk_sample_names, vcf_path, tree_path, f"chr{chr}", exclude_terminus=exclude_terminus)

Processing chr21...


100%|██████████| 512977/512977 [06:08<00:00, 1393.31it/s]


Processing chr22...


100%|██████████| 501421/501421 [06:58<00:00, 1199.15it/s]


In [2]:
chr_list = list(range(11,23))
#chr_list = [21]

exclude_terminus = True

for chr in chr_list:
    print(f"Processing chr{chr}...")
    if exclude_terminus:
        vcf_path = f"{vcf_dir}/graph_chr{chr}_no_terminus.vcf" 
    else:
        vcf_path = f"{vcf_dir}/graph_chr{chr}.vcf"
    
    vcf_df, meta_info = read_vcf_to_dataframe(vcf_path, return_meta_info=True)

    meta_info.insert(-1, '##INFO=<ID=NIA,Number=0,Type=Flag,Description="Nearly identical alleles">\n')
    meta_info = ''.join(meta_info)

    def flag_nearly_identical_alleles(x):
        if x['REF'] != '.':
            nearly_identical = nearly_identical_alleles(x['REF'], x['ALT'])
        else:
            nearly_identical = nearly_identical_alleles(get_info_dict(x['INFO'])['OR'], x['ALT'])
        
        if nearly_identical:
            return x['INFO']+"NIA=1"
        else:
            return x['INFO']

    vcf_df['INFO'] = vcf_df.apply(lambda x: flag_nearly_identical_alleles(x), axis=1)

    if exclude_terminus:
        vcf_path_flag = f"{vcf_dir}/flag_NIA_vcf/graph_chr{chr}_no_terminus.vcf"
    else:
        vcf_path_flag = f"{vcf_dir}/flag_NIA_vcf/graph_chr{chr}.vcf"

    with open(vcf_path_flag, 'w', newline='') as file:
        file.write(meta_info)
        vcf_df.to_csv(file, sep='\t', index=False)

Processing chr11...


Processing chr12...
Processing chr13...
Processing chr14...
Processing chr15...
Processing chr16...
Processing chr17...
Processing chr18...
Processing chr19...
Processing chr20...
Processing chr21...
Processing chr22...
